In [1]:
from datetime import timedelta
import datetime 
import os

from nna.fileUtils import read_file_properties_v2,find_filesv2,list_files
from nna.fileUtils import get_audio,str2timestamp,query_audio,findPhoto

import pandas as pd
from pathlib import Path


from IPython.display import display, Image
import pickle

In [2]:
# !ls ../../data/

In [3]:
# file_properties_df=pd.read_pickle("../../data/realdata_v2No_stinchcomb.pkl")
# file_properties_df2=pd.read_pickle("../../data/stinchcomb_dataV1.pkl")
file_properties_df=pd.read_pickle("../../data/allFields_dataV4.pkl")
dataPoints=[]


with open("../../data/imgOnlyDateV1.pkl", 'rb') as f:
    img_only_data=pickle.load(f)

tmp_folder="/home/enis/projects/nna/data/tmp/"

In [4]:
file_properties_df[0:0]

,site_id,locationId,site_name,recorderId,hour_min_sec,year,month,day,region,timestamp,durationSec,timestampEnd


In [5]:
def display_search(location,region,start_time,length,
                   buffer=1800,audioSearch=True,img_search=True,
                   img_only_data=None,
                   tmp_folder="/home/enis/projects/nna/data/tmp/"):
    
    end_time=None  # or datetime object

    #how to name the file 
    file_name="original2"

    # if there is no recording in given timestamp, it searches before and after, 
    # buffer is how far to look in seconds
#     buffer=1800

    # where to save audio files
#     tmp_folder="/home/enis/projects/nna/data/tmp/"

    if audioSearch:
        output=query_audio(location,region,start_time,end_time,length,buffer,file_properties_df,file_name,display_flag=True,save=True,tmp_folder=tmp_folder)
        if len(output.index)>0:
            print(output.iloc[0].timestamp,"---",output.iloc[0].durationSec)
    if isinstance(start_time,datetime.datetime):
        timestamp = start_time
    else:
        timestamp=datetime.datetime.strptime(start_time, '%Y-%m-%d_%H:%M:%S')
    buffer=datetime.timedelta(minutes=5)
    if img_search:
        imgTime,imgPath=findPhoto(location,timestamp,img_only_data,buffer=buffer)

        if imgTime!=-1:
            display(Image(imgPath,width=600))
        else:
            imgPath='img not found'
    else:
        imgPath='image search off'
    dataPoint="'{},{},{},{}',".format(location,start_time,length,imgPath)
    
    if audioSearch:
        return len(output.index)



In [6]:
# # Query by filename
# excerptName = "46_18871_exact_2019-07-07_13:39:23_0m_0s__0m_10s.flac"
# location = excerptName.split("_")[0]
# region = 'anwr'
# start_time = "_".join(excerptName.split("_")[3:5])
# print(excerptName.replace(":","")[:-6])
# lenght = 30 # in seconds , if set end_time is IGNORED

# #
# display_search(location,region,start_time,lenght,
#                    buffer=1800,audioSearch=True,img_search=True,
#                    img_only_data=img_only_data,
#                    tmp_folder=tmp_folder)

In [259]:
start_time='2019-09-11_00:00:00' # YYYY-MM-DD_HH:MM:SS or datetime object
start_time=datetime.datetime.strptime(start_time, '%Y-%m-%d_%H:%M:%S')

# location ID, examples: AR03', '15', 'SINP07', '24', 'AR08'     file_properties_df["site_id"].unique()
location="03"
# raw_embed_name="/scratch/enis/data/nna/real/prudhoe/26/2019/S4A10255_20190507_073000_vgg/S4A10255_20190507_073000_embeddings000.npy"
region= 'dalton'


In [273]:
# Query by hand

length=30 # in seconds , if set end_time is IGNORED
count=0
while count==0:
    start_time+=datetime.timedelta(minutes=10)
    count = display_search(location,region,start_time,length,
                       buffer=600,audioSearch=True,img_search=False,
                       img_only_data=img_only_data,
                       tmp_folder=tmp_folder)

...


2019-09-11 18:40:00 --- 300.0


In [274]:
# Query by hand

length=30 # in seconds , if set end_time is IGNORED
count=0
while count==0:
    start_time+=datetime.timedelta(minutes=10)
    count = display_search(location,region,start_time,length,
                       buffer=600,audioSearch=True,img_search=False,
                       img_only_data=img_only_data,
                       tmp_folder=tmp_folder)
    

...


2019-09-11 20:00:00 --- 300.0


In [9]:
#Build query by file name
aa = (Path('/tank/data/nna/real/anwr/50/2019/S4A10366_20190513_161602.flac'),
  1353)
start_time=file_properties_df.loc[aa[0]].timestamp+datetime.timedelta(seconds=aa[1])
loc=file_properties_df.loc[aa[0]].locationId
region = file_properties_df.loc[aa[0]].region
lenght = 30 

display_search(loc,region,start_time,lenght,
                   buffer=1800,audioSearch=True,img_search=False,
                   img_only_data=img_only_data,
                   tmp_folder="/home/enis/projects/nna/data/tmp/")


...


original2_exact_2019-05-13_16:38:35
2019-05-13 16:16:02 --- 4438.0
'50,2019-05-13 16:38:35,30,image search off',


In [7]:
# start_time='2019-05-11_07:00:00'




In [64]:

myNotes=\
"""
There is a bird (maybe raven), rocks are moving around
Just a little bit of wind, nothing else
Just wind for 5 minutes
Bird sounds, there is a strong sound mixed with wind,nothing on the image(ice is melted)
Bird sounds, engine like sound mixed with wind,nothing on the image(ice is melted)
There is a storm, no visual, before midnight and after all day, wind and cable is true, bu there are no birds. Wind sounds like some animal screaming
Predictions says there are birds, but it is just wind
There is a static sound that model thinks it is running water, that is kind of my mistake, there are birds so correctly predicted
There is a plane -predicted(low)-, no water wrong prediction, there are few bird sounds
Around midnight there are many water bird sounds
I can here both kinds of birds, -predicted-
I would label this as rain, but it might be just thin cables
I hear waterbirds and wind but prediction is aircraft,songbird wind
"""
dataPoints=\
['31,2019-05-04_06:30:00,30,/tank/data/nna/cameraTrap/anwr/31/2019/100RECNX/RCNX0264.JPG',
 '31,2019-05-04_09:30:00,30,/tank/data/nna/cameraTrap/anwr/31/2019/100RECNX/RCNX0303.JPG',
 '31,2019-05-04_14:30:00,300,/tank/data/nna/cameraTrap/anwr/31/2019/100RECNX/RCNX0363.JPG',
 '31,2019-05-23_14:30:00,30,/tank/data/nna/cameraTrap/anwr/31/2019/100RECNX/RCNX6072.JPG',
 '31,2019-05-23_15:31:00,30,/tank/data/nna/cameraTrap/anwr/31/2019/100RECNX/RCNX6084.JPG',
 '31,2019-05-06_23:50:00,30,/tank/data/nna/cameraTrap/anwr/31/2019/100RECNX/RCNX1138.JPG',
 "31,2019-05-09_08:00:00,30,/tank/data/nna/cameraTrap/anwr/31/2019/100RECNX/RCNX1890.JPG",
 '31,2019-05-10_08:00:00,30,/tank/data/nna/cameraTrap/anwr/31/2019/100RECNX/RCNX2178.JPG',
 '31,2019-05-15_15:59:10,30,/tank/data/nna/cameraTrap/anwr/31/2019/100RECNX/RCNX3768.JPG',
 '31,2019-05-15_00:23:10,30,/tank/data/nna/cameraTrap/anwr/31/2019/100RECNX/RCNX3527.JPG',
 '31,2019-06-10_23:50:00,30,/tank/data/nna/cameraTrap/anwr/31/2019/101RECNX/RCNX1375.JPG',
 '31,2019-06-16_22:25:00,30,/tank/data/nna/cameraTrap/anwr/31/2019/101RECNX/RCNX3998.JPG',
 '31,2019-06-05_01:00:00,30,/tank/data/nna/cameraTrap/anwr/31/2019/100RECNX/RCNX9657.JPG',
]
print(len(myNotes.split("\n"))-2,len(dataPoints))

13 13


In [18]:
import shutil
shutil.rmtree(tmp_folder)

In [58]:
print(dataPoint)


31,2019-05-09_08:00:00,30,/tank/data/nna/cameraTrap/anwr/31/2019/100RECNX/RCNX1890.JPG
